# 5DEEP - Projet final : Classification des chants d'oiseaux

Pour rappel:

Il s'agit d'un problème de classification de chants d'oiseaux avec 5 classes, une pour chacune des espèces suivantes : 

- Bewick's Wren
- Northern Cardinal
- American Robin
- Song Sparrow
- Northern Mockingbird

Pour répondre à cette problématique, nous utiliserons des réseaux de neuronnes que nous préparerons et entraînerons grâce à Keras et Librosa dans ce notebook Jupyter.

## Etape 1: Analyse exploratoire et comprehension des données

Nous allons tout d'abord commencer par comprendre les différentes données mises à disposition, mais aussi comprendre l'interpretation de ces fichiers audio et leur traitement.

In [ ]:
import pandas as pd

bird_df = pd.read_csv("data/bird_songs_metadata.csv")

bird_df.head(10)

bird_df.info()
bird_df.isnull().sum()

print("Nombre d'observations:", len(bird_df))

# On retire 'subspecies' car trop corrélée avec 'species' et pas de sens pour notre classification
# 'recordist', 'license', 'remarks', 'source_url' n'apporte pas de sens pour notre classificiation

# On conserve le 'filename' pour retrouver l'enregistrement associé


# est-ce qu'on garde 'sound_type' ??

# Notre variable cilble est 'species'

cols = ["id", "genus", "species", "name", "country", "location", 
        "latitude", "longitude", "altitude", "time", "date", "filename", 'sound_type']
bird_df_useful = bird_df[cols]

bird_df_useful.head(10)

In [ ]:
## Distribution & corrélations

import matplotlib.pyplot as plt

bird_df_useful['species'].value_counts().plot(kind='bar', figsize=(8,4))
plt.title("Nombre d'échantillons par espèce")
plt.xlabel("Espèce")
plt.ylabel("Nombre d'enregistrements")
plt.show()


In [ ]:
## Test librosa

import librosa
import librosa.display
import numpy as np

sample_file = bird_df_useful['filename'].iloc[0]
path = f"data/wavfiles/{sample_file}"
y, sr = librosa.load(path, sr=22050, mono=True)

# forme du signal
plt.figure(figsize=(10,3))
librosa.display.waveshow(y, sr=sr)
plt.title(f"Waveform - {sample_file}")
plt.show()

# log-mel spectrogram
mel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
log_mel = librosa.power_to_db(mel, ref=np.max)

plt.figure(figsize=(10,4))
librosa.display.specshow(log_mel, sr=sr, hop_length=512, x_axis="time", y_axis="mel")
plt.colorbar(format="%+2.0f dB")
plt.title("Log-Mel Spectrogram")
plt.show()
